In [ ]:
import hvplot.pandas
import holoviews as hv
import numpy as np
import pandas as pd
import panel as pn
from holoviews.plotting.links import RangeToolLink
from sklearn.linear_model import LinearRegression
import datetime
#from mast.utils import string

pd.options.plotting.backend = 'holoviews'
hv.extension('bokeh')


%run functions.ipynb

In [ ]:
# TODO nei reader con read_csv:
# date_parser --> Deprecated since version 2.0.0: Use date_format instead, or read in as object and then apply to_datetime() as-needed.

In [ ]:
def airs_read(filename):
    airs_data = pd.read_csv(filename,header=7)
    airs_data['time'] = pd.to_datetime(airs_data['time'])
    airs_data.rename(columns={'time':'Time',
                              ' mean_AIRS3STD_7_0_TotH2OVap_D':'IWV[mm]',
                              ' AIRS3STD_7_0_TotH2OVap_A':'IWV[mm]'},
                   inplace=True)
    airs_data = df_drop(airs_data, airs_data['IWV[mm]'] < 0.)
    airs_data.Time = airs_data.Time.dt.tz_localize(None)
    airs_data.set_index('Time',drop=False,inplace=True)
    return airs_data

In [ ]:
def cimel_read(filename):
    cimel_data = pd.read_csv(filename,header=[6],delimiter = ',')
    cimel_data.rename(columns={'Date(dd:mm:yyyy) Time(hh:mm:ss)':'Time',
                              'Precipitable_Water(cm)':'IWV[mm]'},inplace=True)    
    cimel_data['Time'] = pd.to_datetime(cimel_data['Time'], 
                                utc=True, format="%d:%m:%Y %H:%M:%S")
    
    cimel_data = df_drop(cimel_data, cimel_data['IWV[mm]'] <0.)
    cimel_data['IWV[mm]'] = cimel_data['IWV[mm]']*10.
    cimel_data['Triplet_Variability_Precipitable_Water(mm)'] = cimel_data['Triplet_Variability_Precipitable_Water(cm)']*10.
    cimel_data.Time = cimel_data.Time.dt.tz_localize(None)
    cimel_data.set_index('Time',drop=False,inplace=True)
    return cimel_data[['Time','IWV[mm]',
                             'Triplet_Variability_Precipitable_Water(mm)']]

In [ ]:
def ftir_read(filename):
    ftir_data = pd.read_csv(filename, header=6)
    ftir_data['Time'] = pd.to_datetime(ftir_data[' YYYY-MM-DD'] +
                                       ftir_data[' HH:MM:SS'])
    ftir_data.rename(columns={' TC [mm]':'IWV[mm]'}, inplace=True)
    ftir_data = df_drop(ftir_data, ftir_data['IWV[mm]'] <0.)
    ftir_data.Time = ftir_data.Time.dt.tz_localize(None)
    ftir_data.set_index('Time',drop=False,inplace=True)
    ftir_data.drop(columns=['Index',' YYYY-MM-DD',' HH:MM:SS'],
                   inplace=True)
    return ftir_data

In [ ]:
def gnss_read(filename, year):
    gnss_data = pd.read_csv(filename,header=[0,1],delim_whitespace=True)
    gnss_data.rename(columns={'PWV':'IWV[mm]'}, inplace=True)
    gnss_data = df_drop(gnss_data,gnss_data['IWV[mm]']['[mm]'] < 0.)

    gnss_data['Time'] = gnss_data.DOY.applymap(lambda x: datetime.datetime(
        int(year),1,1,0,0,0)+datetime.timedelta(days=x-1.))
    gnss_data.drop('DOY',axis=1,inplace=True)
    gnss_data.Time = gnss_data.Time.dt.tz_localize(None)
    gnss_data = gnss_data.set_index('Time', drop=False)

    return gnss_data

In [ ]:
def hatpro_read(fname, year='2016'):
    hatpro = pd.read_csv(fname, delim_whitespace=True, header=0)
    hatpro['Time'] = hatpro.JD_2016.to_frame().applymap(lambda x: 
                                                        datetime.datetime(
                                                            2016,1,1,0,0,0)+
                                                        datetime.timedelta(days=x-1.))
    hatpro.drop(columns=['JD_rif', 'JD_2016','n_med','RF_med'], inplace=True)
    hatpro.rename(columns={'IWV':'IWV[mm]'}, inplace=True)

    hatpro.Time = hatpro.Time.dt.tz_localize(None)
    hatpro.set_index('Time', drop=True,inplace=True)
    hatpro.dropna(subset='IWV[mm]',inplace=True)
    return hatpro

In [ ]:
def modis_read(fname):
    path_laads = fname
    modis = pd.read_csv(path_laads,header=0,sep='\t')
    modis['time'] = pd.to_datetime(modis['time'], utc=True)
    modis['Time'] = modis['time'].dt.tz_localize(None)
    modis = modis.drop(columns='time')
    modis = df_drop(modis, modis.longitude < -65000.)
    modis["IWV_MODIS"] = modis["IWV_MODIS"]*10.
    modis['Err_IWV_MODIS'] = modis["IWV_MODIS"]*0.13
    modis = modis.set_index('Time',drop=False)
    return modis

In [ ]:
# NB questo sembra che non funzioni: da le date riferite al 1970, anche se non da il warning
def radiosounding_read(filename):
    radio_data = pd.read_csv(filename,header=0,sep='\t')
    radio_data.rename(columns={'filename':'Time','PWV[mm]':'IWV[mm]'},
                      inplace=True)
    radio_data['Time'] = pd.to_datetime(radio_data['Time'], utc=True,
                                           format='%Y%m%d%H%M')
    radio_data['Time'] = radio_data['Time'].dt.tz_localize(None)
    radio_data = radio_data.set_index('Time',drop=False)
    return radio_data

In [ ]:
def vespa_read(fname):
    thaao = pd.read_csv(fname, sep="\t", header=5, 
                      names=["Time","IWV_THAAO"])
    thaao['Time'] = pd.to_datetime(thaao['Time'])
    thaao = thaao.set_index('Time', drop=False)
    thaao['err_IWV'] = thaao['IWV_THAAO']*0.05
    return thaao